# GeoLiAs #

Using [geemap](https://geemap.org/) to analyze lineaments generated from Landsat 7 and SRTM. Edges (lineaments) were detected using [Canny Edge Detector](https://developers.google.com/earth-engine/apidocs/ee-algorithms-cannyedgedetector?hl=en) and further analyzed through [Hough Transform](https://developers.google.com/earth-engine/apidocs/ee-algorithms-houghtransform?hl=en).

Layer tab in top right can turn on/off layers as well as change opacity.

To start, just click submit in the bottom to get produce some lineaments.

Change the thresholds and sigma and click submit to change the visualization.

Data Sources -
* [SRTM](https://developers.google.com/earth-engine/datasets/catalog/NASA_NASADEM_HGT_001?hl=en) DEM (blue)
* [Landsat 7](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1?hl=en) (red)

In [1]:
# Import earth engine, geemap, ipywidgets

import ee
import geemap
import ipywidgets as widgets
from datetime import date
geemap.ee_initialize()

today = str(date.today())

Map = geemap.Map()

# Load country boundaries
ctryBnd = ee.FeatureCollection("FAO/GAUL/2015/level0")

Map.addLayer(ctryBnd,
            {},
            'Countries',
            False,
            )

Map


Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [2]:
# Define data sources

dataSources = {
    "Landsat 7": "LANDSAT/LE07/C01/T1",
    "Landsat 8": "LANDSAT/LC08/C01/T1",
    "Landsat 9": "LANDSAT/LC09/CO2/T1_L2",
    "Shuttle Reconnaissance and Topography Mission": "NASA/NASADEM_HGT/001",
    "USGS NED (10m DEM)": "USGS/3DEP/10m",
}

In [3]:
# Load Landsat 7 data, filter by date and bounds.
LS7collection = ee.ImageCollection("LANDSAT/LE07/C01/T1") \
  .filterDate('2018-01-01', today) \
  .filter(ee.Filter.calendarRange(6, 9, 'month'))

# Also filter the LS7collection by the IMAGE_QUALITY property.
LS7filtered = LS7collection \
  .filterMetadata('IMAGE_QUALITY', 'equals', 9)

# Create two composites to check the effect of filtering by IMAGE_QUALITY.
LS7goodComposite = ee.Algorithms \
  .Landsat.simpleComposite(LS7filtered, 75, 3)

# Add PC band
LS7Panchromatic = LS7goodComposite \
  .select(['B8'])

Map.addLayer(LS7Panchromatic,
             {'gain': 1.5},
             'LS7Panchromatic',
             False)

# Add SRTM 30m (https://developers.google.com/earth-engine/datasets/catalog/NASA_NASADEM_HGT_001?hl=en)
SRTM = ee.Image('NASA/NASADEM_HGT/001') \
  .select('elevation')

In [4]:
# Load Landsat 8 data, filter by date and bounds.
LS8collection = ee.ImageCollection("LANDSAT/LC08/C01/T1") \
  .filterDate('2010-01-01', today) \
  .filter(ee.Filter.calendarRange(6, 9, 'month'))

# Also filter the LS8collection by the IMAGE_QUALITY property.
LS8filtered = LS8collection \
# filtering returns no collection? very weird
#  .filterMetadata('IMAGE_QUALITY', 'greater_than', 3)

# Create two composites to check the effect of filtering by IMAGE_QUALITY.
LS8goodComposite = ee.Algorithms \
  .Landsat.simpleComposite(LS8filtered, 75, 3)

# Add PC band
LS8Panchromatic = LS8goodComposite \
  .select(['B8'])

Map.addLayer(LS8Panchromatic,
             {'gain': 1.5},
             'LS8Panchromatic',
             False)

# Add SRTM 30m (https://developers.google.com/earth-engine/datasets/catalog/NASA_NASADEM_HGT_001?hl=en)
SRTM = ee.Image('NASA/NASADEM_HGT/001') \
  .select('elevation')

In [5]:
style = {'description_width': 'initial'}

### Threshold ###
The pixel is only considered for edge detection if the gradient magnitude is higher than this threshold.

In [6]:
#############Old Thresh Declaration###########

# LSthresh = widgets.BoundedIntText(
#   value=10,
#   min=0,
#   max=100,
#   step=1,
#   description='Landsat Threshold',
#   disabled=False,
#   style=style
# )

# DEMthresh = widgets.BoundedIntText(
#   value=10,
#   min=0,
#   max=100,
#   step=1,
#   description='DEM Threshold',
#   disabled=False,
#   style=style
# )

# syncThresh = widgets.Checkbox(
#   value=False,
#   description='Sync Thresholds?',
#   style=style
# )

# hbox1 = widgets.HBox([LSthresh, DEMthresh])
# hbox1

### Sigma ###
Sigma value for a gaussian filter applied before edge detection. 0 means apply no filtering.

In [7]:
#############Old Sigma Declaration###########

# LSsigma = widgets.BoundedFloatText(
#   value=1.5,
#   min=0,
#   max=10,
#   step=0.1,
#   description='Landsat Sigma',
#   disabled=False,
#   style=style
# )

# DEMsigma = widgets.BoundedFloatText(
#   value=1.5,
#   min=0,
#   max=10,
#   step=0.1,
#   description='DEM Sigma',
#   disabled=False,
#   style=style
# )

# syncSigma = widgets.Checkbox(
#   value=False,
#   description='Sync Sigmas?',
#   style=style
# )

# hbox2 = widgets.HBox([LSsigma, DEMsigma])
# hbox2

In [8]:
#############Threshold and Sigma Inputs###########
threshText = widgets.BoundedFloatText(
  description='Threshold',
  layout=widgets.Layout(width='auto', height='auto'),
  min=0,
  max=100,
  step=0.5,
  value=10)

sigmaText = widgets.BoundedFloatText(
  description='Sigma',
  layout=widgets.Layout(width='auto', height='auto'),
  min=0,
  max=50,
  step=0.1,
  value=1.5)

threshSlider = widgets.FloatSlider(
  description='Threshold',
  layout=widgets.Layout(width='auto', height='auto'),
  readout_format='.1f',
  min=0,
  max=100,
  step=0.5,
  value=10)

sigmaSlider = widgets.FloatSlider(
  description='Sigma',
  layout=widgets.Layout(width='auto', height='auto'),
  readout_format='.1f',
  min=0,
  max=50,
  step=0.1,
  value=1.5)

threshSigmaInput = widgets.TwoByTwoLayout(top_left=threshText, top_right=sigmaText, bottom_left=threshSlider, bottom_right=sigmaSlider, width="40%")

link_left = widgets.jslink((threshSigmaInput.top_left, 'value'), (threshSigmaInput.bottom_left, 'value'))
link_right = widgets.jslink((threshSigmaInput.top_right, 'value'), (threshSigmaInput.bottom_right, 'value'))

In [9]:
threshSigmaInput

TwoByTwoLayout(children=(BoundedFloatText(value=10.0, description='Threshold', layout=Layout(grid_area='top-le…

In [10]:
leftMap = widgets.RadioButtons(
    options=[('Landsat 7', 'LANDSAT/LE07/C01/T1'),
    ('Landsat 8', 'LANDSAT/LC08/C01/T1'),
    ('Shuttle Reconnaissance and Topography Mission', 'NASA/NASADEM_HGT/001'),
    ('USGS NED (10m DEM)', 'USGS/3DEP/10m')],
    description='Left Map Data Source',
    disabled=False
)

rightMap = widgets.RadioButtons(
    options=[('Landsat 7', 'LANDSAT/LE07/C01/T1'),
    ('Landsat 8', 'LANDSAT/LC08/C01/T1'),
    ('Shuttle Reconnaissance and Topography Mission', 'NASA/NASADEM_HGT/001'),
    ('USGS NED (10m DEM)', 'USGS/3DEP/10m')],
    description='Right Map Data Source',
    disabled=False
)

dualMap = widgets.TwoByTwoLayout(top_left=leftMap, top_right=rightMap)

In [11]:
dualMap

TwoByTwoLayout(children=(RadioButtons(description='Left Map Data Source', layout=Layout(grid_area='top-left'),…

In [12]:
def single_edge_detect(image, LandsatThresh, LandsatSigma):

    #############Edge Detection###########

    # Detect edges in the panchromatic composite. Variables are (image, threshold, sigma)

    cannyLS = ee.Algorithms \
      .CannyEdgeDetector(image, LandsatThresh, LandsatSigma)

    # Mask the image with itself to get rid of areas with no edges.
    cannyLS = cannyLS \
      .updateMask(cannyLS)

    Map.addLayer(cannyLS, {'min': 0, 'max': 1, 'palette': '0620DB'}, 'CE Pan', False)

    #############Hough Transformation###########

    # Perform Hough transform of the Canny result and display.
    houghLS = ee.Algorithms \
      .HoughTransform(cannyLS, 256, 64, 300, True)

    Map.addLayer(houghLS, {'palette': '1293DB'}, 'Hough', True, 0.5)

    #############Buffer###########
    # Seems to just congest things. Not needed.

    # bufferSize = 250
    # houghLSbuffer = houghLS \
    #   .focal_max(bufferSize, 'square', 'meters')


    # Map.addLayer(houghLSbuffer, {'palette': '1293DB'}, 'Hough LS w/Buffer', False, 0.5)

In [20]:
def edge_detect(LandsatThresh, SRTMThresh, LandsatSigma, SRTMSigma):

    #############Edge Detection###########

    # Detect edges in the panchromatic composite. Variables are (image, threshold, sigma)

    cannyLS = ee.Algorithms \
      .CannyEdgeDetector(LS7Panchromatic, LandsatThresh, LandsatSigma)
    cannyDEM = ee.Algorithms \
      .CannyEdgeDetector(SRTM, SRTMThresh, SRTMSigma)

    # Mask the image with itself to get rid of areas with no edges.
    cannyLS = cannyLS \
      .updateMask(cannyLS)
    cannyDEM = cannyDEM \
      .updateMask(cannyDEM)

    Map.addLayer(cannyLS, {'min': 0, 'max': 1, 'palette': '0620DB'}, 'CE Pan LS', False)
    Map.addLayer(cannyDEM, {'min': 0, 'max': 1, 'palette': '8F1713'}, 'CE DEM', False)

    #############Hough Transformation###########

    # Perform Hough transform of the Canny result and display.
    houghLS = ee.Algorithms \
      .HoughTransform(cannyLS, 256, 64, 300, True)
    houghDEM = ee.Algorithms \
      .HoughTransform(cannyDEM, 256, 64, 300, True)

    # Split map between two sources
    houghLS = geemap.ee_tile_layer(houghLS, {'palette': '1293DB'}, 'LS7')
    houghDEM = geemap.ee_tile_layer(houghDEM, {'palette': 'DB3832'}, 'DEM')
    Map.split_map(houghLS, houghDEM)

    # Map.addLayer(houghLS, {'palette': '1293DB'}, 'Hough LS', True, 0.5)
    # Map.addLayer(houghDEM, {'palette': 'DB3832'}, 'Hough DEM', True, 0.5)
    
    #############Buffer###########
    # Seems to just congest things. Not needed.

    # bufferSize = 250
    # houghLSbuffer = houghLS \
    #   .focal_max(bufferSize, 'square', 'meters')
    # houghDEMbuffer = houghDEM \
    #   .focal_max(bufferSize, 'square', 'meters')

    # Map.addLayer(houghLSbuffer, {'palette': '1293DB'}, 'Hough LS w/Buffer', False, 0.5)
    # Map.addLayer(houghDEMbuffer, {'palette': 'DB3832'}, 'Hough DEM w/Buffer', False, 0.5)

In [14]:
submit = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click the submit the request to create lineaments',
    style=style
)

output = widgets.Output()

In [15]:
def submit_clicked(b):

    with output:
        output.clear_output()
        print('Computing...')
        # file = open("../assets/806.gif")
        # imageLoading = file.read()
        # widgets.Image(
        #     value=imageLoading,
        #     format='gif',
        #     width=80,
        #     height=80,
        # )

        # edge_detect(LandsatThresh=LSthresh.value, SRTMThresh=DEMthresh.value,
        #             LandsatSigma=LSsigma.value, SRTMSigma=DEMsigma.value)
        edge_detect(LandsatThresh=threshText.value, SRTMThresh=threshText.value,
                    LandsatSigma=sigmaText.value, SRTMSigma=sigmaText.value)

submit.on_click(submit_clicked)

In [16]:
submit

Button(button_style='primary', description='Submit', style=ButtonStyle(), tooltip='Click the submit the reques…

In [17]:
output

Output()